In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/miniforge3/envs/nlp-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v3.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "NF-UNSW-NB15-v2.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2151027
1      91904
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    2295222
1      95053
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [16]:
# X = data.drop(columns=["Attack", "Label"])
# y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X = test_data.drop(columns=["Attack", "Label"])
y = test_data[["Attack", "Label"]]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=13, stratify=y)
##

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
220614,59.166.0.4,149.171.126.1,4.802078e-09,3.048648e-09,3.994566e-05,7.652425e-07,0.000072,1.071339e-06,1.980229e-09,1.980360e-09,...,0.000000e+00,1.329685e-03,1.329685e-03,4.043553e-10,4.043553e-10,6.314168e-09,6.314212e-09,0.000000,3.618463e-09,"[4.802078220820267e-09, 3.0486475517277593e-09..."
329844,59.166.0.1,149.171.126.8,9.025364e-09,8.052833e-09,1.079121e-04,8.300931e-07,0.000063,8.300931e-07,1.397062e-08,1.396955e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,1.181795e-08,1.182599e-08,8.561578e-09,8.561639e-09,0.000000,9.271523e-09,"[9.025363504432674e-09, 8.052833119952719e-09,..."
1093665,59.166.0.3,149.171.126.8,3.068685e-13,8.437668e-13,1.638983e-07,3.061234e-09,0.000010,7.129838e-09,1.188386e-13,1.274923e-13,...,6.875550e-09,6.372843e-07,7.080937e-08,4.545956e-14,5.191931e-14,4.034960e-13,4.034988e-13,0.000000,4.369547e-13,"[3.0686849809527964e-13, 8.437667985448175e-13..."
1764268,59.166.0.4,149.171.126.0,2.361402e-08,2.106949e-08,7.927293e-05,1.085931e-06,0.000097,1.085931e-06,3.655283e-08,3.655002e-08,...,0.000000e+00,0.000000e+00,0.000000e+00,3.092057e-08,3.094159e-08,1.878071e-08,3.577436e-10,0.000033,2.425808e-08,"[2.3614022984160315e-08, 2.1069487814953574e-0..."
1525176,59.166.0.9,149.171.126.5,2.309362e-10,2.856102e-10,1.717880e-05,2.649686e-07,0.000124,2.796890e-07,8.943290e-11,9.594530e-11,...,7.360238e-08,1.598644e-04,1.065762e-04,5.174884e-12,5.174884e-12,3.036539e-10,3.036561e-10,0.000000,3.288336e-10,"[2.309362039695701e-10, 2.856102367918858e-10,..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph
train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
train_g = train_g.to_directed()
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))

/tmp/ipykernel_75231/1319261381.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dgi.load_state_dict(torch.load('best_dgi_UNSW_v3.pkl'))


<All keys matched successfully>

In [32]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [33]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [34]:
# Convert to GPU
test_g = test_g.to('cuda')

In [35]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [36]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [37]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,0.059930,-0.232742,-0.005069,-0.024865,0.121915,0.113855,0.156352,-0.056625,0.017936,0.178155,...,0.090383,0.072441,-0.039967,-0.123838,-0.03437,-0.071308,-0.072903,0.101382,Benign,0
1,0.059930,-0.232742,-0.005069,-0.024865,0.121915,0.113855,0.156352,-0.056625,0.017936,0.178155,...,0.090383,0.072441,-0.039967,-0.123838,-0.03437,-0.071308,-0.072903,0.101382,Benign,0
2,0.059930,-0.232742,-0.005069,-0.024865,0.121915,0.113855,0.156352,-0.056625,0.017936,0.178155,...,0.090383,0.072441,-0.039967,-0.123838,-0.03437,-0.071308,-0.072903,0.101382,Benign,0
3,0.059930,-0.232742,-0.005069,-0.024865,0.121915,0.113855,0.156352,-0.056625,0.017936,0.178155,...,0.090383,0.072441,-0.039967,-0.123838,-0.03437,-0.071308,-0.072903,0.101382,Benign,0
4,0.059930,-0.232742,-0.005069,-0.024865,0.121915,0.113855,0.156352,-0.056625,0.017936,0.178155,...,0.090383,0.072441,-0.039967,-0.123838,-0.03437,-0.071308,-0.072903,0.101382,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334631,0.112737,-0.005271,0.119856,0.011221,-0.002853,0.043843,0.063991,0.040263,-0.023027,0.221056,...,0.044236,0.045262,-0.003596,0.092189,0.00999,-0.018107,-0.003408,0.084470,Benign,0
334632,0.112737,-0.005271,0.119856,0.011221,-0.002853,0.043843,0.063991,0.040263,-0.023027,0.221056,...,0.044236,0.045262,-0.003596,0.092189,0.00999,-0.018107,-0.003408,0.084470,Benign,0
334633,0.112737,-0.005271,0.119856,0.011221,-0.002853,0.043843,0.063991,0.040263,-0.023027,0.221056,...,0.044236,0.045262,-0.003596,0.092189,0.00999,-0.018107,-0.003408,0.084470,Benign,0
334634,0.112737,-0.005271,0.119856,0.011221,-0.002853,0.043843,0.063991,0.040263,-0.023027,0.221056,...,0.044236,0.045262,-0.003596,0.092189,0.00999,-0.018107,-0.003408,0.084470,Benign,0


# Embeddings CBLOF  Embeddings

In [38]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [39]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [40]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,0.059753,-0.232270,-0.005008,-0.024660,0.121689,0.113519,0.156137,-0.056262,0.018003,0.178350,...,0.017751,0.003806,0.090492,0.072474,-0.039442,-0.123612,-0.034442,-0.071516,-0.072937,0.101727
1,0.059753,-0.232270,-0.005008,-0.024660,0.121689,0.113519,0.156137,-0.056262,0.018003,0.178350,...,0.017751,0.003806,0.090492,0.072474,-0.039442,-0.123612,-0.034442,-0.071516,-0.072937,0.101727
2,0.059753,-0.232270,-0.005008,-0.024660,0.121689,0.113519,0.156137,-0.056262,0.018003,0.178350,...,0.017751,0.003806,0.090492,0.072474,-0.039442,-0.123612,-0.034442,-0.071516,-0.072937,0.101727
3,0.059753,-0.232270,-0.005008,-0.024660,0.121689,0.113519,0.156137,-0.056262,0.018003,0.178350,...,0.017751,0.003806,0.090492,0.072474,-0.039442,-0.123612,-0.034442,-0.071516,-0.072937,0.101727
4,0.059753,-0.232270,-0.005008,-0.024660,0.121689,0.113519,0.156137,-0.056262,0.018003,0.178350,...,0.017751,0.003806,0.090492,0.072474,-0.039442,-0.123612,-0.034442,-0.071516,-0.072937,0.101727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143413,0.128318,-0.023319,0.169950,0.018355,-0.018010,0.022430,0.014558,-0.017123,0.017344,0.278179,...,0.063008,-0.003047,0.063919,0.081291,-0.004493,0.118635,0.032591,0.037440,0.033387,0.023689
143414,0.128318,-0.023319,0.169950,0.018355,-0.018010,0.022430,0.014558,-0.017123,0.017344,0.278179,...,0.063008,-0.003047,0.063919,0.081291,-0.004493,0.118635,0.032591,0.037440,0.033387,0.023689
143415,0.128318,-0.023319,0.169950,0.018355,-0.018010,0.022430,0.014558,-0.017123,0.017344,0.278179,...,0.063008,-0.003047,0.063919,0.081291,-0.004493,0.118635,0.032591,0.037440,0.033387,0.023689
143416,0.128318,-0.023319,0.169950,0.018355,-0.018010,0.022430,0.014558,-0.017123,0.017344,0.278179,...,0.063008,-0.003047,0.063919,0.081291,-0.004493,0.118635,0.032591,0.037440,0.033387,0.023689


In [41]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [42]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
220614,4.802078e-09,3.048648e-09,3.994566e-05,7.652425e-07,0.000072,1.071339e-06,1.980229e-09,1.980360e-09,0.000000e+00,0.000000,...,1.329685e-03,1.329685e-03,4.043553e-10,4.043553e-10,6.314168e-09,6.314212e-09,0.000000,3.618463e-09,Benign,0
329844,9.025364e-09,8.052833e-09,1.079121e-04,8.300931e-07,0.000063,8.300931e-07,1.397062e-08,1.396955e-08,1.452568e-08,0.000000,...,0.000000e+00,0.000000e+00,1.181795e-08,1.182599e-08,8.561578e-09,8.561639e-09,0.000000,9.271523e-09,Benign,0
1093665,3.068685e-13,8.437668e-13,1.638983e-07,3.061234e-09,0.000010,7.129838e-09,1.188386e-13,1.274923e-13,0.000000e+00,0.000042,...,6.372843e-07,7.080937e-08,4.545956e-14,5.191931e-14,4.034960e-13,4.034988e-13,0.000000,4.369547e-13,Benign,0
1764268,2.361402e-08,2.106949e-08,7.927293e-05,1.085931e-06,0.000097,1.085931e-06,3.655283e-08,3.655002e-08,3.800509e-08,0.000000,...,0.000000e+00,0.000000e+00,3.092057e-08,3.094159e-08,1.878071e-08,3.577436e-10,0.000033,2.425808e-08,Benign,0
1525176,2.309362e-10,2.856102e-10,1.717880e-05,2.649686e-07,0.000124,2.796890e-07,8.943290e-11,9.594530e-11,0.000000e+00,0.000000,...,1.598644e-04,1.065762e-04,5.174884e-12,5.174884e-12,3.036539e-10,3.036561e-10,0.000000,3.288336e-10,Benign,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1525437,1.940855e-09,1.232171e-09,1.123952e-04,2.041300e-06,0.000155,2.165015e-06,8.003488e-10,8.004016e-10,0.000000e+00,0.000000,...,1.253977e-03,1.074837e-03,1.006141e-10,1.006141e-10,2.551995e-09,2.552013e-09,0.000000,0.000000e+00,Benign,0
1636406,2.617265e-08,2.335241e-08,7.823357e-05,1.203593e-06,0.000097,1.203593e-06,4.051340e-08,4.051029e-08,4.212302e-08,0.000000,...,0.000000e+00,0.000000e+00,3.427088e-08,3.429417e-08,2.053465e-08,3.965058e-10,0.000036,2.688649e-08,Benign,0
152133,4.439600e-10,5.490674e-10,5.999424e-06,1.131967e-07,0.000125,1.697950e-07,1.719290e-10,3.618107e-09,0.000000e+00,0.000000,...,1.434202e-04,8.195440e-05,8.057850e-10,8.063327e-10,5.837552e-10,5.837593e-10,0.000000,6.321614e-10,Benign,0
1164696,2.451807e-09,1.556554e-09,9.775598e-05,1.797272e-06,0.000137,1.953557e-06,1.011050e-09,1.011116e-09,0.000000e+00,0.000000,...,1.244650e-03,1.131500e-03,4.425931e-10,4.423147e-10,3.223837e-09,3.223860e-09,0.000000,0.000000e+00,Benign,0


In [43]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [44]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [45]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:05<00:00,  1.83s/it]

{'n_estimators': 9, 'con': 0.01}
0.8333306487827521
              precision    recall  f1-score   support

           0     0.9849    0.9904    0.9876    137714
           1     0.7324    0.6329    0.6790      5704

    accuracy                         0.9762    143418
   macro avg     0.8586    0.8117    0.8333    143418
weighted avg     0.9748    0.9762    0.9754    143418



In [46]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:06<00:00,  1.84s/it]

{'n_estimators': 2, 'con': 0.05}
0.8690922253962816
              precision    recall  f1-score   support

           0     0.9900    0.9890    0.9895    137714
           1     0.7399    0.7577    0.7487      5704

    accuracy                         0.9798    143418
   macro avg     0.8649    0.8733    0.8691    143418
weighted avg     0.9800    0.9798    0.9799    143418



In [47]:
###  CBLOF RAW

In [48]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:06,  5.25it/s]

2
2


 11%|█         | 4/36 [00:00<00:05,  5.41it/s]

2
2


 17%|█▋        | 6/36 [00:01<00:05,  5.33it/s]

2
2


 28%|██▊       | 10/36 [00:02<00:08,  2.96it/s]

3


100%|██████████| 36/36 [00:14<00:00,  2.55it/s]

{'n_estimators': 3, 'con': 0.04}
0.7183189342694002
              precision    recall  f1-score   support

           0     0.9793    0.9716    0.9755     68857
           1     0.4244    0.5049    0.4612      2852

    accuracy                         0.9531     71709
   macro avg     0.7019    0.7383    0.7183     71709
weighted avg     0.9573    0.9531    0.9550     71709



In [49]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:05,  5.95it/s]

2
2


 11%|█         | 4/36 [00:00<00:06,  5.30it/s]

2
2


 14%|█▍        | 5/36 [00:00<00:05,  5.27it/s]

2


 19%|█▉        | 7/36 [00:01<00:06,  4.14it/s]

3


100%|██████████| 36/36 [00:14<00:00,  2.54it/s]

benign only
{'n_estimators': 3}
0.7173358973597662
              precision    recall  f1-score   support

           0     0.9793    0.9714    0.9753     68857
           1     0.4218    0.5042    0.4594      2852

    accuracy                         0.9528     71709
   macro avg     0.7006    0.7378    0.7173     71709
weighted avg     0.9571    0.9528    0.9548     71709



In [50]:
# HBOS  Embeddings

In [51]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [52]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [02:15<00:00,  3.76s/it]

{'n_estimators': 20, 'con': 0.01}
0.9000775446669387
              precision    recall  f1-score   support

           0     0.9945    0.9884    0.9915    137714
           1     0.7568    0.8682    0.8087      5704

    accuracy                         0.9837    143418
   macro avg     0.8757    0.9283    0.9001    143418
weighted avg     0.9851    0.9837    0.9842    143418



In [53]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:20<00:00,  3.90s/it]

{'n_estimators': 5, 'con': 0.05}
0.9264385476143018
              precision    recall  f1-score   support

           0     1.0000    0.9865    0.9932    137714
           1     0.7539    1.0000    0.8597      5704

    accuracy                         0.9870    143418
   macro avg     0.8769    0.9932    0.9264    143418
weighted avg     0.9902    0.9870    0.9879    143418



In [54]:
##  HBOS  RAw

In [55]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:12<00:00,  2.94it/s]

{'n_estimators': 30, 'con': 0.04}
0.7146257915510277
              precision    recall  f1-score   support

           0     0.9820    0.9613    0.9715     68857
           1     0.3805    0.5743    0.4577      2852

    accuracy                         0.9459     71709
   macro avg     0.6812    0.7678    0.7146     71709
weighted avg     0.9581    0.9459    0.9511     71709



In [56]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:11<00:00,  3.06it/s]

benign only
{'n_estimators': 30}
0.6688907552042004
              precision    recall  f1-score   support

           0     0.9740    0.9719    0.9730     68857
           1     0.3556    0.3745    0.3648      2852

    accuracy                         0.9481     71709
   macro avg     0.6648    0.6732    0.6689     71709
weighted avg     0.9494    0.9481    0.9488     71709



In [57]:
##  PCA  Emb

In [58]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [01:02<00:00,  1.75s/it]

{'n_estimators': 5, 'con': 0.01}
0.8651245892314753
              precision    recall  f1-score   support

           0     0.9892    0.9895    0.9893    137714
           1     0.7438    0.7381    0.7409      5704

    accuracy                         0.9795    143418
   macro avg     0.8665    0.8638    0.8651    143418
weighted avg     0.9794    0.9795    0.9794    143418



In [59]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:07<00:00,  1.89s/it]

{'n_estimators': 5, 'con': 0.05}
0.8906426200100054
              precision    recall  f1-score   support

           0     0.9932    0.9884    0.9908    137714
           1     0.7496    0.8361    0.7905      5704

    accuracy                         0.9824    143418
   macro avg     0.8714    0.9123    0.8906    143418
weighted avg     0.9835    0.9824    0.9828    143418



In [60]:
##  PCA  RAw

In [61]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:08,  3.89it/s]

100%|██████████| 36/36 [00:10<00:00,  3.43it/s]

{'n_estimators': 15, 'con': 0.04}
0.707439747737215
              precision    recall  f1-score   support

           0     0.9815    0.9598    0.9705     68857
           1     0.3671    0.5628    0.4444      2852

    accuracy                         0.9440     71709
   macro avg     0.6743    0.7613    0.7074     71709
weighted avg     0.9570    0.9440    0.9496     71709



In [62]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:10<00:00,  3.52it/s]

benign only
{'n_estimators': 5}
0.6589707768342642
              precision    recall  f1-score   support

           0     0.9843    0.9222    0.9522     68857
           1     0.2554    0.6441    0.3657      2852

    accuracy                         0.9111     71709
   macro avg     0.6198    0.7832    0.6590     71709
weighted avg     0.9553    0.9111    0.9289     71709



In [63]:
##  IF  Emb

In [64]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:40<00:00,  1.70s/it]

{'n_estimators': 100, 'con': 0.01}
0.8333582998638966
              precision    recall  f1-score   support

           0     0.9843    0.9916    0.9879    137714
           1     0.7523    0.6183    0.6788      5704

    accuracy                         0.9767    143418
   macro avg     0.8683    0.8050    0.8334    143418
weighted avg     0.9751    0.9767    0.9756    143418



In [65]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [00:41<00:00,  1.73s/it]

{'n_estimators': 50, 'con': 0.05}
0.9264385476143018
              precision    recall  f1-score   support

           0     1.0000    0.9865    0.9932    137714
           1     0.7539    1.0000    0.8597      5704

    accuracy                         0.9870    143418
   macro avg     0.8769    0.9932    0.9264    143418
weighted avg     0.9902    0.9870    0.9879    143418



In [66]:
##  IF  Raw

In [67]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 24/24 [00:16<00:00,  1.47it/s]

{'n_estimators': 50, 'con': 0.1}
0.6760553178862567
              precision    recall  f1-score   support

           0     0.9924    0.9056    0.9470     68857
           1     0.2676    0.8327    0.4051      2852

    accuracy                         0.9027     71709
   macro avg     0.6300    0.8692    0.6761     71709
weighted avg     0.9636    0.9027    0.9255     71709



In [68]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:16<00:00,  1.50it/s]

benign only
{'n_estimators': 100}
0.6668448354584398
              precision    recall  f1-score   support

           0     0.9854    0.9234    0.9534     68857
           1     0.2657    0.6690    0.3803      2852

    accuracy                         0.9133     71709
   macro avg     0.6255    0.7962    0.6668     71709
weighted avg     0.9567    0.9133    0.9306     71709

